# SAM: Animation Inference Playground

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'SAM'

In [ ]:
!git clone https://github.com/yuval-alaluf/SAM.git $CODE_DIR

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

In [ ]:
os.chdir(f'./{CODE_DIR}')

In [ ]:
from argparse import Namespace
import os
import sys
import pprint
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from datasets.augmentations import AgeTransformer
from utils.common import tensor2im
from models.psp import pSp

In [ ]:
EXPERIMENT_TYPE = 'ffhq_aging'

## Step 1: Download Pretrained Model
As part of this repository, we provide our pretrained aging model.
We'll download the model for the selected experiments as save it to the folder `../pretrained_models`.

In [ ]:
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../pretrained_models. """
    current_directory = os.getcwd()
    save_path = os.path.join(os.path.dirname(current_directory), "pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url

In [ ]:
MODEL_PATHS = {
    "ffhq_aging": {"id": "1XyumF6_fdAxFmxpFcmPf-q84LU_22EMC", "name": "sam_ffhq_aging.pt"}
}

path = MODEL_PATHS[EXPERIMENT_TYPE]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])

In [ ]:
!wget {download_command}

## Step 3: Define Inference Parameters

Below we have a dictionary defining parameters such as the path to the pretrained model to use and the path to the
image to perform inference on.
While we provide default values to run this script, feel free to change as needed.

In [ ]:
EXPERIMENT_DATA_ARGS = {
    "ffhq_aging": {
        "model_path": "../pretrained_models/sam_ffhq_aging.pt",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

In [ ]:
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[EXPERIMENT_TYPE]

## Step 4: Load Pretrained Model
We assume that you have downloaded the pretrained aging model and placed it in the path defined above.

In [ ]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')

In [ ]:
opts = ckpt['opts']
pprint.pprint(opts)

In [ ]:
# update the training options
opts['checkpoint_path'] = model_path

In [ ]:
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

### Utils for Generating MP4 

In [ ]:
import imageio
from tqdm import tqdm
import matplotlib
from IPython.display import HTML
from base64 import b64encode

matplotlib.use('module://ipykernel.pylab.backend_inline')
%matplotlib inline


def generate_mp4(out_name, images, kwargs):
    writer = imageio.get_writer(out_name + '.mp4', **kwargs)
    for image in images:
        writer.append_data(image)
    writer.close()


def run_on_batch_to_vecs(inputs, net):
    _, result_batch = net(inputs.to("cuda").float(), return_latents=True, randomize_noise=False, resize=False)
    return result_batch.cpu()


def get_result_from_vecs(vectors_a, vectors_b, alpha):
    results = []
    for i in range(len(vectors_a)):
        cur_vec = vectors_b[i] * alpha + vectors_a[i] * (1 - alpha)
        res = net(cur_vec.cuda(), randomize_noise=False, input_code=True, input_is_full=True, resize=False)
        results.append(res[0])
    return results


def show_mp4(filename, width=400):
    mp4 = open(filename + '.mp4', 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video width="%d" controls autoplay loop>
        <source src="%s" type="video/mp4">
    </video>
    """ % (width, data_url)))

In [ ]:
SEED = 42
np.random.seed(SEED)

img_transforms = EXPERIMENT_ARGS['transform']
n_transition = 25
kwargs = {'fps': 40}
save_path = "notebooks/animations"
os.makedirs(save_path, exist_ok=True)

#################################################################
# TODO: define your image paths here to be fed into the model
#################################################################
root_dir = 'notebooks/images'
ims = ['866', '1287', '2468']
im_paths = [os.path.join(root_dir, im) + '.jpg' for im in ims]

# NOTE: Please make sure the images are pre-aligned!

target_ages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 0]
age_transformers = [AgeTransformer(target_age=age) for age in target_ages]

for image_path in im_paths:
    image_name = os.path.basename(image_path)
    print(f'Working on image: {image_name}')
    original_image = Image.open(image_path).convert("RGB")
    input_image = img_transforms(original_image)
    all_vecs = []
    for idx, age_transformer in enumerate(age_transformers):

        input_age_batch = [age_transformer(input_image.cpu()).to('cuda')]
        input_age_batch = torch.stack(input_age_batch)

        # get latent vector for the current target age amount
        with torch.no_grad():
            result_vec = run_on_batch_to_vecs(input_age_batch, net)
            result_image = get_result_from_vecs([result_vec], result_vec, 0)[0]
            all_vecs.append([result_vec])

    images = []
    for i in range(1, len(target_ages)):
        alpha_vals = np.linspace(0, 1, n_transition).tolist()
        for alpha in tqdm(alpha_vals):
            result_image = get_result_from_vecs(all_vecs[i-1], all_vecs[i], alpha)[0]
            output_im = tensor2im(result_image)
            images.append(np.array(output_im))

    animation_path = os.path.join(save_path, f"{image_name}_animation")
    generate_mp4(animation_path, images, kwargs)
    show_mp4(animation_path)